In [96]:
import pandas as pd
import numpy as np
import random
import toolsets.data_prep as data_prep
%load_ext autoreload
%autoreload 2
descriptors = pd.read_csv("data/multiRT/MultiRT_hilic_descriptors.csv", low_memory=False)
descriptors = descriptors.select_dtypes(exclude=['object'])

i am updated!
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
combined_data = pd.read_csv("data/multiRT/Combined dataset.csv")
hilic = combined_data.loc[combined_data['Column']=='HILIC']
hilic.reset_index(inplace=True, drop=True)
# hilic=data_prep.dataset_prep(hilic)
# hilic = data_prep.make_split_index(hilic)

In [103]:
data = pd.concat([hilic, descriptors],axis =1)

In [104]:
data = data_prep.dataset_prep(data)

In [105]:
train, test = data_prep.make_train_test(data, 0.8,0.2)

In [106]:
train.head()

,Organic_modifier,pH,Buffer,retention_time,ABC,ABCGG,nAcid,nBase,SpAbs_A,SpMax_A,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,Acetonitrile,2.7,Formic acid,1.822857,14.008885,10.537147,0,0,23.372976,2.54742,...,10.190582,51.083298,218.073165,8.076784,428,33,100.0,125.0,4.027778,3.583333
1,Acetonitrile,3.0,Acetic acid,12.321429,14.008885,10.537147,0,0,23.372976,2.54742,...,10.190582,51.083298,218.073165,8.076784,428,33,100.0,125.0,4.027778,3.583333
2,Acetonitrile,3.0,Formic acid,1.857143,14.008885,10.537147,0,0,23.372976,2.54742,...,10.190582,51.083298,218.073165,8.076784,428,33,100.0,125.0,4.027778,3.583333
3,Acetonitrile,5.0,Acetic acid,1.850000,14.008885,10.537147,0,0,23.372976,2.54742,...,10.190582,51.083298,218.073165,8.076784,428,33,100.0,125.0,4.027778,3.583333
4,Acetonitrile,5.0,Formic acid,1.851429,14.008885,10.537147,0,0,23.372976,2.54742,...,10.190582,51.083298,218.073165,8.076784,428,33,100.0,125.0,4.027778,3.583333


In [69]:
hilic = hilic.drop(['Compound_name', 'Column','SMILES'], axis=1)

In [71]:
data.shape

(836, 1331)

In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836 entries, 0 to 835
Columns: 1331 entries, Organic_modifier to mZagreb2
dtypes: bool(2), float64(1007), int64(320), object(2)
memory usage: 8.5+ MB


# start of reproducing clean lab code

In [16]:
SEED = 123456
np.random.seed(SEED)
random.seed(SEED)

In [17]:
from sklearn.datasets import fetch_openml

data = fetch_openml("credit-g")  # get the credit data from OpenML
X_raw = data.data  # features (pandas DataFrame)
y_raw = data.target  # labels (pandas Series)

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [77]:
X_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   checking_status         1000 non-null   category
 1   duration                1000 non-null   float64 
 2   credit_history          1000 non-null   category
 3   purpose                 1000 non-null   category
 4   credit_amount           1000 non-null   float64 
 5   savings_status          1000 non-null   category
 6   employment              1000 non-null   category
 7   installment_commitment  1000 non-null   float64 
 8   personal_status         1000 non-null   category
 9   other_parties           1000 non-null   category
 10  residence_since         1000 non-null   float64 
 11  property_magnitude      1000 non-null   category
 12  age                     1000 non-null   float64 
 13  other_payment_plans     1000 non-null   category
 14  housing                 1

In [72]:
from sklearn.preprocessing import StandardScaler

In [79]:
cat_features = X_raw.select_dtypes("category").columns
X_encoded = pd.get_dummies(X_raw, columns=cat_features, drop_first=True)

In [81]:
num_features = X_raw.select_dtypes("float64").columns
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[num_features] = scaler.fit_transform(X_encoded[num_features])

y = y_raw.map({"bad": 0, "good": 1})  # encode labels as integers

In [86]:
X_raw['duration']

0       6.0
1      48.0
2      12.0
3      42.0
4      24.0
       ... 
995    12.0
996    30.0
997    12.0
998    45.0
999    45.0
Name: duration, Length: 1000, dtype: float64

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
from sklearn.model_selection import cross_val_predict

num_crossval_folds = 3  # for efficiency; values like 5 or 10 will generally work better
pred_probs = cross_val_predict(
    clf,
    X_scaled,
    y,
    cv=num_crossval_folds,
    method="predict_proba",
)